In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import json
import pymongo
from src.benchmark.utils import get_system_info

print("PyMongo Version:", pymongo.__version__)
print("System Info:\n", json.dumps(get_system_info(), indent=3))

PyMongo Version: 4.13.1
System Info:
 {
   "python_version": "3.12.1",
   "system": {
      "os": "Darwin",
      "release": "24.5.0",
      "machine": "x86_64",
      "processor": "i386"
   },
   "cpu": {
      "physical_cores": 8,
      "total_cores": 8,
      "frequency_mhz": 2400
   },
   "memory": {
      "total_ram_gb": 16.0,
      "available_ram_gb": 0.88
   }
}


# Data Load

In [4]:
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")

In [5]:
try:
    client.admin.command('ping')
    print("Successfully connected to MongoDB")
except Exception as e:
    print(f"Connection failed: {e}")

Successfully connected to MongoDB


In [6]:
print("Databases:", client.list_database_names())

Databases: ['admin', 'bdnr_MovieLens_ml_25m', 'benchmark_db', 'config', 'local']


In [7]:
from pprint import pprint

def print_collection_info(db_name, collection_name):
    print(f"\n{db_name}.{collection_name}")
    collection = client[db_name][collection_name]
    for doc in collection.find().limit(1):  # First documents
        pprint(doc)

# Admin DB collections
print_collection_info('admin', 'system.version')
# Local DB collections
print_collection_info('local', 'startup_log')


admin.system.version
{'_id': 'featureCompatibilityVersion', 'version': '7.0'}

local.startup_log
{'_id': 'UY000MC64N6TY7K-1747265527787',
 'buildinfo': {'allocator': 'system',
               'bits': 64,
               'buildEnvironment': {'cc': '/Applications/Xcode13.2.1.app/Contents/Developer/Toolchains/XcodeDefault.xctoolchain/usr/bin/clang: '
                                          'Apple clang version 13.0.0 '
                                          '(clang-1300.0.29.30)',
                                    'ccflags': '-Wno-error=unknown-warning-option '
                                               '-isysroot '
                                               '/Applications/Xcode13.2.1.app/Contents/Developer/Platforms/MacOSX.platform/Developer/SDKs/MacOSX12.1.sdk '
                                               '-mmacosx-version-min=11.0 '
                                               '--target=darwin20.0.0 -arch '
                                               'arm64 -Werro

### Create new collections

In [8]:
# Create database and collections
db = client["bdnr_MovieLens_ml_25m"]

movies_col = db["movies"]
users_col = db["users"]

In [9]:
# client.drop_database("bdnr_MovieLens_ml_25m_test")

### Load collections

In [10]:
import pandas as pd

# Read CSV files
base_path = "/Users/efaliveni//Downloads/ml-25m"

ratings = pd.read_csv(f"{base_path}/ratings.csv")
movies = pd.read_csv(f"{base_path}/movies.csv")
tags = pd.read_csv(f"{base_path}/tags.csv")
genome_tags = pd.read_csv(f"{base_path}/genome-tags.csv")
genome_scores = pd.read_csv(f"{base_path}/genome-scores.csv")

In [11]:
# Filter Data
SAMPLE_SIZE = 500

selected_users = ratings['userId'].unique()[:SAMPLE_SIZE]
filtered_ratings = ratings[ratings['userId'].isin(selected_users)]
filtered_movie_ids = filtered_ratings['movieId'].unique()
filtered_movies = movies[movies['movieId'].isin(filtered_movie_ids)]
filtered_tags = tags[tags['movieId'].isin(filtered_movie_ids)]
filtered_scores = genome_scores[genome_scores['movieId'].isin(filtered_movie_ids)]

In [12]:
from collections import defaultdict
# Create a dictionary of genome tags for quick lookup
genome_tags_dict = {row['tagId']: row['tag'] for _, row in genome_tags.iterrows()}

# Build dictionaries to accumulate data
ratings_by_movie = defaultdict(list)
scores_by_movie = defaultdict(list)

ratings_by_user = defaultdict(list)
tags_by_user = defaultdict(list)

In [ ]:
# Process ratings and tags to build both movie and user views
RELEVANCE_THRESHOLD = 0.8

# Ratings
for _, r in filtered_ratings.iterrows():
    movie_id = int(r.movieId)
    user_id = int(r.userId)
    
    # Add to movie's ratings
    ratings_by_movie[movie_id].append({
        "userId": user_id,
        "rating": float(r.rating),
        "timestamp": int(r.timestamp)
    })
    
    # Add to user's ratings
    ratings_by_user[user_id].append({
        "movieId": movie_id,
        "rating": float(r.rating),
        "timestamp": int(r.timestamp)
    })
    
# Tags (only for users), genome tag is the cleaned infered tag
for _, t in filtered_tags.dropna(subset=["tag"]).iterrows():
    movie_id = int(t.movieId)
    user_id = int(t.userId)
    
    tags_by_user[user_id].append({
        "movieId": movie_id,
        "tag": t.tag,
        "timestamp": int(t.timestamp)
    })
    
# Genome scores (only for movies), with 0.8 threshold
for _, s in filtered_scores[filtered_scores.relevance > RELEVANCE_THRESHOLD].iterrows():
    movie_id = int(s.movieId)
    tag_id = int(s.tagId)
    
    scores_by_movie[movie_id].append({
        "tagId": tag_id,
        "tag": genome_tags_dict.get(tag_id, ""),
        "relevance": float(s.relevance)
    })

In [ ]:
import re

# Build movie documents
movie_docs = []
for _, m in filtered_movies.iterrows():
    movie_id = int(m.movieId)
   
    # Extract year from title
    title = m.title
    year_match = re.search(r"\((\d{4})\)$", title)
    year = int(year_match.group(1)) if year_match else None
    
    # Calculate statistics
    movie_ratings = ratings_by_movie.get(movie_id, [])
    rating_count = len(movie_ratings)
    if rating_count > 0:
        avg_rating = sum(r['rating'] for r in movie_ratings) / rating_count
        # Calculate rating distribution (count of each star rating)
        rating_dist = {"0.5":0, "1.0":0, "1.5":0, "2.0":0, "2.5":0, 
                      "3.0":0, "3.5":0, "4.0":0, "4.5":0, "5.0":0}
        for r in movie_ratings:
            rating_key = f"{r['rating']}"
            rating_dist[rating_key] += 1 
    else:
        avg_rating = 0.0
        rating_dist = {}
        
    movie_docs.append({
        "movieId": movie_id,
        "title": title,
        "year": year,
        "genres": m.genres.split("|") if pd.notna(m.genres) else [],
        "ratings": ratings_by_movie.get(movie_id, []),
        "tagGenome": scores_by_movie.get(movie_id, []),
        "stats": {
            "ratingCount": rating_count,
            "avgRating": round(avg_rating, 2),
            "ratingDistribution": rating_dist
        },

    })

In [ ]:
# Build user documents with rating interaction history
user_docs = []
for user_id in selected_users:
    user_id = int(user_id)
    
    # Get all user activity
    user_ratings = ratings_by_user.get(user_id, [])
    user_tags = tags_by_user.get(user_id, [])
    
    # Calculate basic stats
    rating_count = len(user_ratings)
    avg_rating = sum(r['rating'] for r in user_ratings) / rating_count if rating_count > 0 else 0
    tag_count = len(user_tags)
    
    # Create rating interaction history
    rating_interactions = [
        {
            "movieId": rating['movieId'],
            "rating": rating['rating'],
            "timestamp": rating['timestamp']
        }
        for rating in user_ratings
    ]
    
    # Add tags to history with type designation
    # tag_interactions = [
    #     {
    #         "movieId": tag['movieId'],
    #         "tag": tag['tag'],
    #         "timestamp": tag['timestamp']
    #     }
    #     for tag in user_tags
    # ]
    
    # Sort interactions by timestamp (oldest first)
    rating_interactions.sort(key=lambda x: x['timestamp'])
    # tag_interactions.sort(key=lambda x: x['timestamp'])
    
    user_docs.append({
        "userId": user_id,
        "stats": {
            "ratingCount": rating_count,
            "avgRating": round(avg_rating, 2),
        },
        "ratingHistory": rating_interactions,
        # "tagHistory": tag_interactions
    })

In [ ]:
# movies_col.delete_many({})
# users_col.delete_many({})

In [ ]:
import time

# Insert data
start = time.time()
movies_col.insert_many(movie_docs)
users_col.insert_many(user_docs)
end = time.time()

print(f"Total processing and loading time: {end - start:.2f} seconds")
print(f"Movies loaded: {len(movie_docs)}")
print(f"Users loaded: {len(user_docs)}")

### Sample Movie

In [ ]:
sample_movie = movies_col.find_one({}, {'_id': 0})  # Exclude MongoDB _id
print(json.dumps(sample_movie, indent=2, default=str))

### Sample User

In [ ]:
sample_user = users_col.find_one({}, {'_id': 0}) 
print(json.dumps(sample_user, indent=2, default=str))

# Single-Threaded Benchmark
Simulate User Load, real time latency stress

In [ ]:
### Define Test Queries - Read Operations

# queries = [
#     ("Find by ID", lambda: movies_col.find_one({"movieId": 1})),
#     ("Top Rated", lambda: list(movies_col.find({"stats.avgRating": {"$gt": 4.5}}).limit(10))),
#     ("Genre Search", lambda: list(movies_col.find({"genres": "Comedy"}).limit(20))),
#     ("User History", lambda: list(users_col.find({"stats.ratingCount": {"$gt": 100}}))),
#     ("Aggregate Ratings", lambda: list(movies_col.aggregate([
#         {"$match": {"year": {"$gte": 2000}}},
#         {"$group": {"_id": None, "avgRating": {"$avg": "$stats.avgRating"}}}
#     ]))),
#     ("Title Search (Regex)", lambda: list(movies_col.find(
#         {"title": {"$regex": "story", "$options": "i"}}  # Case-insensitive regex search
#     ).limit(5))),
#     ("User Ratings with Movie Details", lambda: list(users_col.aggregate([
#          {"$match": {"userId": {"$in": [int(x) for x in selected_users[:5]]}}},
#          {"$lookup": {
#              "from": "movies",
#              "localField": "ratings.movieId",
#              "foreignField": "movieId",
#              "as": "ratedMovies"
#          }},
#          {"$project": {
#              "userId": 1,
#              "ratedMovies": {
#                  "$map": {
#                      "input": "$ratedMovies",
#                      "as": "movie",
#                      "in": {
#                          "title":     "$$movie.title",
#                          "year":      "$$movie.year",
#                          "genres":    "$$movie.genres",
#                          "avgRating": "$$movie.stats.avgRating"
#                      }
#                  }
#              },
#              "ratingCount": {"$size": {"$ifNull":["$ratings",[]]}}
#          }},
#          {"$limit": 5}
#      ])))
# ]

In [ ]:
# query_name, benchmark_query = queries[1][0], queries[1][1]
# explain()
movies_col.find({"movieId": 1}).limit(1).explain()

### Benchmark:

- Prevent memory caching.
- Use indexed queries that simulate realistic application behavior.
- Measure read & writes.
- Randomized access patterns, multi-threading, or batch loads. (Out of scope)


In [ ]:
# Create Index to measure indexed vs. non-indexed performance:
# movies_col.create_index("movieId")
movies_col.create_index("genres")
users_col.create_index("userId")

In [ ]:
from src.benchmark.utils import timed_op

@timed_op
def perform_read(col, movie_id):
    return list(col.find({"movieId": movie_id}).limit(1))

@timed_op
def perform_write(col, doc):
    return col.insert_one(doc)

In [ ]:
import random
import time
import itertools
from pymongo import MongoClient
from src.benchmark.mongo_listener import BenchListener
from src.benchmark.mongo_benchmark import run_benchmark


GENRES = [
    "Adventure", "Animation", "Children", "Comedy", "Fantasy", "Action",
    "Drama", "Horror", "Sci-Fi", "Romance", "Thriller"
]

TAG_VOCAB = [
    "animated", "cgi", "classic", "pixar", "disney", "heartwarming", "kids",
    "fun", "friendship", "great movie", "story", "nostalgic", "imdb top 250"
]

RATING_VALUES = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, 5.0]


def generate_random_movie_doc(movie_id: int) -> dict:
    num_ratings = random.randint(30, 120)
    ratings = [
        {
            "userId": random.randint(1, 1_000),
            "rating": round(random.choice(RATING_VALUES), 1),
            "timestamp": int(time.time()) - random.randint(0, 10**7)
        }
        for _ in range(num_ratings)
    ]

    # Rating distribution
    rating_distribution = {f"{r:.1f}": 0 for r in RATING_VALUES}
    for r in ratings:
        rating_distribution[f"{r['rating']:.1f}"] += 1

    avg_rating = round(sum(r["rating"] for r in ratings) / len(ratings), 2)

    return {
        "movieId": movie_id,
        "title": f"Random Movie {movie_id}",
        "year": random.randint(1950, 2025),
        "genres": random.sample(GENRES, k=random.randint(2, 5)),
        "ratings": ratings,
        "tagGenome": [
            {
                "tagId": tid,
                "tag": TAG_VOCAB[tid % len(TAG_VOCAB)],
                "relevance": round(random.uniform(0.75, 1.0), 5)
            }
            for tid in [random.randint(1, 1000) for _ in range(random.randint(10, 20))]
        ],
        "stats": {
            "ratingCount": num_ratings,
            "avgRating": avg_rating,
            "ratingDistribution": rating_distribution
        }
    }


listener = BenchListener(watched_commands={"find", "insert"})
client = MongoClient("mongodb://localhost:27017", event_listeners=[listener])
db = client["bdnr_MovieLens_ml_25m"]
movies_col = db["movies"]

# ---- READ benchmark ----
read_query_id = "find_movie_id"
read_query = lambda: perform_read(movies_col, random.randint(1, 1_000))

read_result = run_benchmark(read_query_id, read_query, listener, duration=20)
print("READ result summary:", read_result.summary)

# ---- WRITE benchmark ----
movie_id_counter = itertools.count(start=1_000_001)

write_query_id = "insert_one"
write_query = lambda: perform_write(
    movies_col,
    generate_random_movie_doc(next(movie_id_counter))
)

write_result = run_benchmark("write_query_id", write_query, listener, duration=20)
print("WRITE result summary:", write_result.summary)

In [ ]:
from src.benchmark.utils import plot_benchmark_results

plot_benchmark_results(
    read_query_id,
    read_result.series["sys_cpu_pct"],
    read_result.series["sys_mem_pct"],
    read_result.series["wall_ms"],    
    read_result.series["driver_ms"],
    read_result.series["mongo_rss_mb"]
)

In [ ]:
from src.benchmark.utils import plot_benchmark_results

plot_benchmark_results(
    write_query_id,
    write_result.series["sys_cpu_pct"],
    write_result.series["sys_mem_pct"],
    write_result.series["wall_ms"],    
    write_result.series["driver_ms"],
    write_result.series["mongo_rss_mb"]
)

In [ ]:
# TODOS:
# In each iteration i have to random select the id, otherwise it  will be cahced
# FIX QUERIES to bechmark
# ADD BENCHMARK OF INSERT OPERATIONS

# INDEX Benchmarking?
# Tradeoff entre memory usage y latency
# How is the indexing done? algorithm?

# MongoDB's internal Profiler

Diagnosing MongoDB query behavior, useful for optimizing queries and indexes  

In [ ]:
# Deep dive in server-side execution
def profiled_benchmark():
    db.command('profile', 0)  # Disable first
    db.system.profile.drop()  # Clear old data
    db.command('profile', 2, slowms=0)  # Profile all operations
    
    # Run benchmark
    for _ in range(1):
        movies_col = db["movies"]
        _ = movies_col.find_one({}, {'_id': 0}) 
    
    # Collect profile data
    time.sleep(1)  # Ensure all ops are captured
    profile_data = list(db.system.profile.find({
        "ns": {"$ne": f"{movies_col}.system.profile"},
        "op": {"$in": ["query", "update", "insert", "delete", "command"]},
    }).sort("ts", -1))
    
    analyzed_data = {
        "query_count": len(profile_data),
        "avg_duration": (sum(op.get("millis", 0) for op in profile_data) / 
                       len(profile_data) if profile_data else 0),
        "operations": []
    }
#   print(profile_data)
  
    for op in profile_data[:100]:  # Limit to first 100 ops
        analyzed_data["operations"].append({
            "operation": op.get("op"),
            "namespace": op.get("ns"),
            "duration_ms": op.get("millis", 0),
            "docs_examined": op.get("docsExamined", 0),
            "keys_examined": op.get("keysExamined", 0),
            "nreturned": op.get("nreturned", 0),
            "plan": op.get("planSummary", "N/A")
        })
        
    return analyzed_data

In [ ]:
profiled_benchmark()

# Global server-wide metrics

Server Status Global Info, useful for health dashboards

In [ ]:
# serverStatus

def extract_mongodb_benchmark_metrics(server_status):
    uptime_sec = server_status.get('uptime', 1)
    # MongoDB uses the WiredTiger storage engine (default since v3.2)
    wiredtiger = server_status.get('wiredTiger', {})
    opcounters = server_status.get('opcounters', {})

    
    # Query Efficiency
    query_executor = server_status.get('queryExecutor', {})
    scanned_objects = query_executor.get('scannedObjects', 1)
    index_usage_ratio = (
        (scanned_objects - query_executor.get('scanned', 0)) / 
        max(1, scanned_objects)
    ) * 100  # Percentage of queries using indexes

    # Throughput & Latency
    metrics = {
        'throughput': {
            'reads_sec': opcounters.get('query', 0) / uptime_sec,
            'writes_sec': (
                opcounters.get('insert', 0) + 
                opcounters.get('update', 0) + 
                opcounters.get('delete', 0)
            ) / uptime_sec,
        },
        'latency_ms': {
            'read_avg': server_status.get('opLatencies', {}).get('reads', {}).get('latency', 0) / 1000,
            'write_avg': server_status.get('opLatencies', {}).get('writes', {}).get('latency', 0) / 1000,
            'disk_read_avg': wiredtiger.get('perf', {}).get('file_system_read_latency', 0),
        },
        'index_efficiency': {
            'index_usage_percent': index_usage_ratio,
            'cache_hit_ratio': (
                wiredtiger.get('cache', {}).get('pages_requested', 1) - 
                wiredtiger.get('cache', {}).get('pages_read', 0)
            ) / max(1, wiredtiger.get('cache', {}).get('pages_requested', 1)) * 100,
        },
        'concurrency': {
            'active_connections': server_status.get('connections', {}).get('active', 0),
            'locked_transactions': server_status.get('globalLock', {}).get('currentQueue', {}).get('writers', 0),
        }
    }
    return metrics

server_status = client.admin.command('serverStatus')
pprint(extract_mongodb_benchmark_metrics(server_status))